In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

today = datetime.date.today()

api_link = "https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/summary?startTime="+str(today.year-1)+"-"+str(today.month)+"-"+str(today.day)+"&includeNegativeGeneration=true&format=json"

df = pd.read_json(api_link)

In [32]:
print(df.shape)
df.head()

(367, 3)


,startTime,settlementPeriod,data
0,2023-11-13T00:00:00Z,1,"[{'fuelType': 'BIOMASS', 'generation': 2788}, ..."
1,2023-11-14T00:00:00Z,1,"[{'fuelType': 'BIOMASS', 'generation': 2357}, ..."
2,2023-11-15T00:00:00Z,1,"[{'fuelType': 'BIOMASS', 'generation': 2861}, ..."
3,2023-11-16T00:00:00Z,1,"[{'fuelType': 'BIOMASS', 'generation': 3023}, ..."
4,2023-11-17T00:00:00Z,1,"[{'fuelType': 'BIOMASS', 'generation': 2783}, ..."


In [33]:
df["data"][1]

[{'fuelType': 'BIOMASS', 'generation': 2357},
 {'fuelType': 'CCGT', 'generation': 9517},
 {'fuelType': 'COAL', 'generation': 447},
 {'fuelType': 'INTELEC', 'generation': 50},
 {'fuelType': 'INTEW', 'generation': -128},
 {'fuelType': 'INTFR', 'generation': 984},
 {'fuelType': 'INTIFA2', 'generation': 882},
 {'fuelType': 'INTIRL', 'generation': -321},
 {'fuelType': 'INTNED', 'generation': 577},
 {'fuelType': 'INTNEM', 'generation': 729},
 {'fuelType': 'INTNSL', 'generation': 715},
 {'fuelType': 'NPSHYD', 'generation': 397},
 {'fuelType': 'NUCLEAR', 'generation': 3527},
 {'fuelType': 'OCGT', 'generation': 4},
 {'fuelType': 'OIL', 'generation': 0},
 {'fuelType': 'OTHER', 'generation': 261},
 {'fuelType': 'PS', 'generation': -16},
 {'fuelType': 'WIND', 'generation': 9471}]

In [34]:
pd.json_normalize(df["data"][365]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
fuelType,BIOMASS,CCGT,COAL,INTELEC,INTEW,INTFR,INTGRNL,INTIFA2,INTIRL,INTNED,INTNEM,INTNSL,INTVKL,NPSHYD,NUCLEAR,OCGT,OIL,OTHER,PS,WIND
generation,2612,15115,0,0,-480,1967,0,483,-445,-279,437,1398,-794,196,4737,39,0,490,-85,6747


In [35]:
row = pd.json_normalize(df["data"][1]).T
row.reset_index(inplace=True)
row.pop("index")
row.columns = row.iloc[0]
row.drop(0, inplace=True)
row.reset_index()
row

,BIOMASS,CCGT,COAL,INTELEC,INTEW,INTFR,INTIFA2,INTIRL,INTNED,INTNEM,INTNSL,NPSHYD,NUCLEAR,OCGT,OIL,OTHER,PS,WIND
1,2357,9517,447,50,-128,984,882,-321,577,729,715,397,3527,4,0,261,-16,9471


In [36]:
columns =  pd.json_normalize(df["data"][365]).T.iloc[0].values
new_df = pd.DataFrame(columns=columns)
new_df


,BIOMASS,CCGT,COAL,INTELEC,INTEW,INTFR,INTGRNL,INTIFA2,INTIRL,INTNED,INTNEM,INTNSL,INTVKL,NPSHYD,NUCLEAR,OCGT,OIL,OTHER,PS,WIND


In [37]:
## Unpacking the json format into different columns 
for i in range(len(df)):
    row = pd.json_normalize(df["data"][i]).T
    row.reset_index(inplace=True)
    row.pop("index")
    row.columns = row.iloc[0]
    row.drop(0, inplace=True)
    new_df.loc[len(new_df)] = row.iloc[0]

new_df.fillna(0, inplace=True)
new_df.head()

/tmp/ipykernel_1719875/3654081605.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_df.fillna(0, inplace=True)


,BIOMASS,CCGT,COAL,INTELEC,INTEW,INTFR,INTGRNL,INTIFA2,INTIRL,INTNED,INTNEM,INTNSL,INTVKL,NPSHYD,NUCLEAR,OCGT,OIL,OTHER,PS,WIND
0,2788,5869,353,35,-158,825,0,948,-113,-33,446,695,0,373,3679,4,0,234,-322,13262
1,2357,9517,447,50,-128,984,0,882,-321,577,729,715,0,397,3527,4,0,261,-16,9471
2,2861,11972,839,789,-320,607,0,-2,-317,-132,244,-106,0,523,3493,22,0,310,-139,9797
3,3023,18943,1412,951,-240,960,0,-2,-206,599,772,1237,0,587,3052,147,0,515,143,1270
4,2783,18337,1369,705,-92,598,0,-2,-158,134,450,1065,0,558,3055,24,0,459,27,2153


In [38]:
## Combining all the interconnections into 1 column
inters = columns[3:13]
new_df["INT"] = [new_df.iloc[i].values[3:13].sum() for i in range(len(new_df))]
new_df.head()

,BIOMASS,CCGT,COAL,INTELEC,INTEW,INTFR,INTGRNL,INTIFA2,INTIRL,INTNED,...,INTNSL,INTVKL,NPSHYD,NUCLEAR,OCGT,OIL,OTHER,PS,WIND,INT
0,2788,5869,353,35,-158,825,0,948,-113,-33,...,695,0,373,3679,4,0,234,-322,13262,2645
1,2357,9517,447,50,-128,984,0,882,-321,577,...,715,0,397,3527,4,0,261,-16,9471,3488
2,2861,11972,839,789,-320,607,0,-2,-317,-132,...,-106,0,523,3493,22,0,310,-139,9797,763
3,3023,18943,1412,951,-240,960,0,-2,-206,599,...,1237,0,587,3052,147,0,515,143,1270,4071
4,2783,18337,1369,705,-92,598,0,-2,-158,134,...,1065,0,558,3055,24,0,459,27,2153,2700


In [42]:
# putting all interconnectors in own dataframe and dropping them from new_df#

int_df = new_df[inters]
new_df.drop(inters, axis=1, inplace=True)
int_df.head()

,INTELEC,INTEW,INTFR,INTGRNL,INTIFA2,INTIRL,INTNED,INTNEM,INTNSL,INTVKL
0,35,-158,825,0,948,-113,-33,446,695,0
1,50,-128,984,0,882,-321,577,729,715,0
2,789,-320,607,0,-2,-317,-132,244,-106,0
3,951,-240,960,0,-2,-206,599,772,1237,0
4,705,-92,598,0,-2,-158,134,450,1065,0


In [45]:
new_df.to_csv("./data.csv")
int_df.to_csv("./int_data.csv")